## 배포 및 게시된 Foundry Agent 활용하기

### 새로운 Agent를 생성하고, 버전 추가하기

1. Foundry Portal(Preview)에서 새로운 Agent를 생성합니다.
    - Agent의 이름은 CoffeeExpert라고 주고, 배포 Model은 GPT-4.1로 선택합니다.
    - [저장] 버튼을 클릭하여, 현재 에이전트를 저장합니다.
    - [저장] 버튼 좌측에 **v1 저장됨** 이라고 나오는 것을 확인합니다.
2. Agent의 [지침(Instruction)]란에 다음과 같이 입력합니다.

    ```text
    나는 커피에 대한 전문적인 지식을 제공하는 에이전트입니다. 오직 커피에 대한 정보만을 제공하고, 그 외의 정보는 "죄송합니다. 저는 커피 전문가이다보니 그 밖의 내용은 잘 모르겠습니다."로 대답해야 합니다. 
    ```
    - [저장] 버튼을 클릭하여, 현재 에이전트를 저장합니다.
    - [저장] 버튼 좌측에 **v2 저장됨** 이라고 나오는 것을 확인합니다.
3. 이제, Agent의 우측 상단에 있는 [게시] 버튼을 클릭하여, 현재 Agent를 게시합니다.
    - 게시가 완료되면, [게시됨] 이라고 나오는 것을 확인합니다.
    - 세부 정보 화면이 나오면, 그 중 [응답 API 엔드포인트]를 복사해 둡니다. 이후 코드에서 이 URL을 사용합니다.

HTTP 응답 테스트를 위해서는 임시 Access Token을 발급받아야 합니다. 실제 코드에서는 DefaultAzureCredential 등을 사용하여 Managed Identity 방식으로 인증을 처리할 수 있기에 토큰이 필요없지만, Postman이나 requests 라이브러리를 사용할 때는 토큰이 필요합니다. 하기 코드를 수행하고 accessToken 값을 얻습니다.

In [ ]:
!az account get-access-token --resource https://ai.azure.com

In [ ]:
import requests

publised_url = "<응답 API 엔드포인트>"
accessToken = "<임시 발급받은 액세스 토큰>"

#publised_url = "https://aif-taeyok.services.ai.azure.com/api/projects/proj-default/applications/CoffeeExpert/protocols/openai/responses?api-version=2025-11-15-preview"

headers = {
    "Content-Type":"application/json", 
    "Authorization":f"Bearer {accessToken}"
}

input = {"input":"지금 삼성전자 주식을 사도 될까?"}
response = requests.post(publised_url, headers=headers, json=input)

if(response.status_code != 200):
    print("Error:", response.status_code, response.text)
    exit(1)
else:
    text = response.json()['output'][0]['content'][0]['text']
    print("결과:", text)

#### MAF SDK 사용해서 배포/게시된 Azure AI Foundry Agent 호출하기

게시된 에이전트가 제공하는 API는 현재 [Response API 엔드포인트]를 제공하므로, 그에 따라 AzureOpenAIResponsesClient를 사용합니다.

**(이 부분은 현재 SKIP 요망. 오류 해결 중)**

In [ ]:
import os
from agent_framework.azure import AzureOpenAIResponsesClient, AzureOpenAIChatClient 
from azure.identity import AzureCliCredential

publised_url="https://aif-taeyok.services.ai.azure.com/api/projects/proj-default/applications/CoffeeExpert/protocols/openai/responses"

agent = AzureOpenAIResponsesClient(
    endpoint=publised_url,
    deployment_name=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
    api_version="2025-11-15-preview",
    credential=AzureCliCredential(), 
).as_agent()

result = await agent.run("아메리카노와 롱블랙의 차이는?")
print(result)

# result = await agent.run("지금 삼성전자 주식을 사도 될까?")
# print(result.text)

#### OpenAI SDK 사용해서 배포/게시된 Azure AI Foundry Agent 호출하기

In [ ]:
from openai import OpenAI 
from azure.identity import DefaultAzureCredential, get_bearer_token_provider 

# edit base_url with your <foundry-resource-name>, <project-name>, and <app-name>
openai = OpenAI(
    api_key=get_bearer_token_provider(DefaultAzureCredential(), "https://ai.azure.com/.default"),
    base_url="https://aif-taeyok.services.ai.azure.com/api/projects/proj-default/applications/CoffeeExpert/protocols/openai",
    default_query = {"api-version": "2025-11-15-preview"}
)

response = openai.responses.create( 
  input="아메리카노와 롱블랙의 차이는?", 
) 
print(f"Response output: {response.output_text}")

response = openai.responses.create( 
  input="지금 삼성전자 주식을 사도 될까?", 
) 
print(f"Response output: {response.output_text}")

### Foundry Agent(Preview)를 통해서 호출하기

다음의 코드는 배포된 에이전트를 사용해서 사용자의 요청을 호출하는 방법을 보여줍니다.   
**(신규 포탈에 있는 Agent 호출 방법입니다. Classic 포탈의 Agent 호출 방법은 [AzureAIFoundryAgent.ipynb](AzureAIFoundryAgent.ipynb) 참조)**

In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AgentReference

# Configuration
PROJECT_ENDPOINT =  os.environ["AZURE_AI_PROJECT_ENDPOINT"]
AGENT_NAME = "CoffeeExpert"

# Initialize the client and retrieve the agent
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=DefaultAzureCredential())
agent = client.agents.get(agent_name=AGENT_NAME)

# Get the OpenAI client and send a message
openai_client = client.get_openai_client()
response = openai_client.responses.create(
    input=[{"role": "user", "content": "겨울 아침에 어울리는 커피는?"}],
    extra_body={"agent": AgentReference(name=agent.name).as_dict()}
)

print(f"Agent response: {response.output_text}")


이렇게 Agent를 통해서 호출하게 되면, 모든 대화 이력과 사용량이 Foundry 포탈에 기록되며, 완전한 Observability를 제공합니다.   
Foundry Portal에서 Agent의 [추적] 탭을 통해서 다음과 같이 대화 이력과 사용량을 모니터링할 수 있습니다.  

![MAF-inoke-log1.png](./images/MAF-inoke-log1.png)  

![MAF-inoke-log1.png](./images/MAF-inoke-log2.png)

### 특정 배포버전의 Agent 호출하기

다음의 코드는 배포된 에이전트의 특정 배포버전을 지정하여 호출하는 방법을 보여줍니다.

In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AgentReference

# Configuration
PROJECT_ENDPOINT =  os.environ["AZURE_AI_PROJECT_ENDPOINT"]
AGENT_NAME = "CoffeeExpert"
AGENT_VERSION = "3"  # Optional: specify version, or use latest

# Initialize the client and retrieve the agent
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=DefaultAzureCredential())
agent = client.agents.get(agent_name=AGENT_NAME)

# Get the OpenAI client and send a message
openai_client = client.get_openai_client()
response = openai_client.responses.create(
    input=[{"role": "user", "content": "지금 현대자동차 주식을 사도 될까?"}],
    extra_body={"agent": AgentReference(name=agent.name, version=AGENT_VERSION).as_dict()}
)

print(f"Agent response: {response.output_text}")
